In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3 as lite
import os
import sys
#from matplotlib import pyplot as plt
import numpy as np
import statsmodels.api as sm
#%matplotlib inline
from collections import Counter
import itertools

In [149]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'}

In [151]:
cities = [['tx', "austin"] ,
          [ 'ca' , 'los-angeles'] ,
          [ 'ga' , 'atlanta'] ,
          [ 'ma' , 'boston'] ,
          [ 'nc' , 'charlotte'] ,
          [ 'tx' , 'dallas'] ,
          [ 'co' , 'denver'] ,
          [ 'tx' , 'houston'] ,
          [ 'fl' , 'miami'] ,
          [ 'mn' , 'minneapolis'] ,
          [ 'ny' , 'new-york'] ,
          [ 'pa' , 'philadelphia'] ,
          [ 'az' , 'phoenix'] ,
          [ 'or' , 'portland'] ,
          [ 'ca' , 'san-francisco'] ,
          [ 'ca' , 'san-diego'] ,
          [ 'wa' , 'seattle'] ,
          [ 'dc' , 'washington']]

In [155]:
datadir = 'crawl/'
if not(os.path.isdir(datadir)):
    os.makedirs(datadir)

for city in cities:
    k = city[0]
    v = city[1]
    running = True
    page = 1
    print('')
    print(k,v)
    time.sleep(1)
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        filename = datadir + k + '-' + v + '-' + str(page) + '.htm'
        if not(os.path.isfile(filename)):
            sys.stdout.write('-')
            r = requests.get(url, headers=headers)
            time.sleep(1)
            f = open(filename, 'wb')
            f.write(r.text.encode('ascii', 'replace'))
            f.close()
            data = r.text
        else:
            sys.stdout.write('.')
            f = open(filename, 'r')
            data = f.read()
            f.close()
        soup = BeautifulSoup(data, "lxml")
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
                #print(url)
                #print(pl)
        page+=1


tx austin
..
ca los-angeles
............
ga atlanta
...
ma boston
....
nc charlotte
......
tx dallas
....
co denver
....
tx houston
........
fl miami
.........
mn minneapolis
.........
ny new-york
..........
pa philadelphia
.....
az phoenix
...
or portland
..
ca san-francisco
.....
ca san-diego
....
wa seattle
..
dc washington
.....

In [168]:
# Data extraction phase

times = []
fees = []
reviews = []
repeats = []
city = []
services = []
ratings = []
name = []
#security = []
#instant = []
#dog = []


for src in os.listdir(datadir):
    filename = datadir + src
    #print('Processing: ' + filename)
    f = open(filename, 'r')
    data = f.read()
    f.close()
    soup = BeautifulSoup(data, 'lxml')
    sitters = soup.findAll('div', {'class': 'dv-host-list-item'}) 
    for sitter in sitters:
        rtwrap = sitter.findAll('i', {'class': 'dv-icon dv-icon__clock'})
        responder = sitter.findAll('li', {'class': 'dv-search-badge-item'})
        clock = len(rtwrap)
        if clock > 0:
            response_time = responder[0].text.replace('Responds in', '').strip()
        else:
            response_time = 0
        fee = sitter.findAll('span', {'class': "dv-host-list-item__price__amount"})[0].text.replace('/night', '').strip()[1:]
        review = sitter.find('span', {'class': 'dv-host-review'})
        if review is not None and len(review) > 0:
            review = review.text.replace('Reviews', '').replace('Review', '').replace('Testimonial', '').replace('Testimonials', '').strip()
        else:
            review = 0
        full_star = sitter.findAll('i', {'class': 'dv-icon__star'})
        rating = len(full_star)
        half_star = sitter.findAll('i', {'class': 'dv-icon__star-half'})
        if len(half_star) > 0:
            rating += .5
        repeat = sitter.find('span', {'class': 'dv-host-repeat'})
        if repeat is not None and len(repeat) > 0:
            repeat = repeat.text.strip().replace('Repeat Guests', '').replace('Repeat Guest', '')
        else:
            repeat = 0
        town = soup.findAll('span', {'id': 'search-location'})[0].text.strip()
        service = sitter.findAll('div', {'class': 'dv-host-list-item__info dv-host-list-item__services'})[0].text.strip()
        fees.append(fee)
        reviews.append(review)
        ratings.append(rating)
        repeats.append(repeat)
        city.append(town)
        services.append(service)
        times.append(response_time)
#Dataframe from dict of objects 
df = pd.DataFrame({'fee':fees,
                   'review':reviews,
                   'rating':ratings,
                   'repeat':repeats,
                   'town':city, 
                   'service':services,
                   'response_time': times})

In [169]:
df

,fee,rating,repeat,response_time,review,service,town
0,40,5.0,1,45 minutes,3,"Boarding, Daycare, Walking, Checkups and Visit...","Phoenix, AZ"
1,25,4.5,14,3 hours,17,Boarding,"Phoenix, AZ"
2,25,4.5,3,35 minutes,14,"Boarding, Daycare, & Bathing","Phoenix, AZ"
3,25,5.0,4,20 minutes,12,"Boarding, Daycare, Checkups and Visits, Groomi...","Phoenix, AZ"
4,35,5.0,29,5 minutes,60,"Boarding, Sitting, Daycare, Checkups and Visit...","Phoenix, AZ"
5,25,0.0,0,3 hours,0,"Boarding, Daycare, Walking, Checkups and Visit...","Phoenix, AZ"
6,30,5.0,1,a few days,1,"Boarding, Daycare, Walking, Checkups and Visit...","Phoenix, AZ"
7,25,5.0,2,15 minutes,5,"Boarding, Sitting, Daycare, Walking, & More","Phoenix, AZ"
8,32,5.0,0,2 hours,3,"Boarding, Walking, Checkups and Visits, & Pick...","Phoenix, AZ"
9,25,5.0,10,an hour,22,Boarding & Bathing,"Phoenix, AZ"
